In [1]:
library(stringr)
library(glue)
library(dplyr)
library(tidyr)
library(tidyverse)
library(fastDummies)
options(gsubfn.engine = "R") #handles a subtle sqldf problem (from https://github.com/ggrothendieck/sqldf/blob/master/INSTALL )
library(sqldf)
library(data.table)
library(assertthat)
"%ni%" <- Negate("%in%")

Warning message:
“package ‘stringr’ was built under R version 3.6.3”
Attaching package: ‘dplyr’

The following object is masked from ‘package:glue’:

    collapse

The following objects are masked from ‘package:stats’:

    filter, lag

The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union

Registered S3 methods overwritten by 'ggplot2':
  method         from 
  [.quosures     rlang
  c.quosures     rlang
  print.quosures rlang
Registered S3 method overwritten by 'rvest':
  method            from
  read_xml.response xml2
── Attaching packages ─────────────────────────────────────── tidyverse 1.2.1 ──
✔ ggplot2 3.1.1     ✔ readr   1.3.1
✔ tibble  2.1.1     ✔ purrr   0.3.4
✔ ggplot2 3.1.1     ✔ forcats 0.5.1
Warning message:
“package ‘purrr’ was built under R version 3.6.3”Warning message:
“package ‘forcats’ was built under R version 3.6.3”── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::collapse() masks gl

In [2]:
num_files_input <- 5
num_files_input <- 19
preserve_old_analysis <- FALSE

# read in some sample data
path = "/Users/abhijit/IGRI/data/asf_with_code.csv"
df1 = read.csv(path)
# print(colnames(df1))

path = "/Users/abhijit/IGRI/data/fpc_files/fpc_{i}_with_code.csv"


In [3]:
#df1
nrow(df1)

[1] 7129

In [4]:
df2 <- data.frame()
for (i in 1:num_files_input) {
  df2 <- rbind(df2, read.csv(glue(path)))
}

df2 = df2[ , (names(df2) %in% c("id","sid","tid","word","pos","relation","word_source","CodeType"))]
df1 = df1[ , (names(df1) %in% c("id","sid","tid","word","pos","relation","word_source","CodeType"))]
fpc1 <- rbind(df1,df2)


In [5]:
print(nrow(df2))
print(nrow(df1))


[1] 14939
[1] 7129


In [6]:
##3 slow but easy repair of sid (which in prior code is equal to 1 always --- not doing what it should.  can recover because tid is sound)
l_tid = 0
l_sid = 0
for (i in 1:nrow(fpc1)) {
    if (fpc1[i,"tid"] < l_tid) {
        l_sid = l_sid + 1
    }
    fpc1[i,"sid"] <- l_sid
    l_tid <- fpc1[i,"tid"]
}

In [7]:
length(unique(fpc1$sid))

[1] 891

In [8]:
sid_toolong = c( 26,  87, 117, 296, 446)
#fpc1 = fpc1[df.sid.apply(lambda x : x not in sid_toolong)].reset_index(drop=True)
fpc1 = subset(fpc1, sid %ni% sid_toolong)



In [9]:
##3 slow but easy repair of sid (which in prior code is equal to 1 always --- not doing what it should.  can recover because tid is sound)
l_tid = 0
l_sid = 0
for (i in 1:nrow(fpc1)) {
    if (fpc1[i,"tid"] < l_tid) {
        l_sid = l_sid + 1
    }
    fpc1[i,"sid"] <- l_sid
    l_tid <- fpc1[i,"tid"]
}

In [10]:
length(unique(fpc1$sid))
head(fpc1)

[1] 886

id,sid,tid,word,pos,relation,word_source,CodeType
doc1,0,0,ROOT,,NA,NA,1
doc1,0,1,each,DT,det,podling,1
doc1,0,2,podling,NN,nsubj,report,1
doc1,0,3,in,IN,case,incubation,1
doc1,0,4,incubation,NN,nmod,podling,1
doc1,0,5,must,MD,aux,report,1


In [11]:
# pull just the data we want
fpc1$new_relations <- as.factor(fpc1$relation)
fpc1$new_word <- as.factor(fpc1$word)
fpc1$new_pos <- as.factor(fpc1$pos)
fpc1$new_source <- as.factor(fpc1$word_source)




In [12]:
#setDT(fpc1)[, .(sid_text = toString(unique(word))), by = sid]

sentences <- sqldf("SELECT sid, GROUP_CONCAT(word, ' ') AS sid_text FROM fpc1 GROUP BY sid;")
fpc1 <- merge(fpc1, sentences, by="sid")





head(fpc1)

sid,id,tid,word,pos,relation,word_source,CodeType,new_relations,new_word,new_pos,new_source,sid_text
0,doc1,0,ROOT,,NA,NA,1,NA,ROOT,,NA,ROOT each podling in incubation must report to the incubator pmc .
0,doc1,1,each,DT,det,podling,1,det,each,DT,podling,ROOT each podling in incubation must report to the incubator pmc .
0,doc1,2,podling,NN,nsubj,report,1,nsubj,podling,NN,report,ROOT each podling in incubation must report to the incubator pmc .
0,doc1,3,in,IN,case,incubation,1,case,in,IN,incubation,ROOT each podling in incubation must report to the incubator pmc .
0,doc1,4,incubation,NN,nmod,podling,1,nmod,incubation,NN,podling,ROOT each podling in incubation must report to the incubator pmc .
0,doc1,5,must,MD,aux,report,1,aux,must,MD,report,ROOT each podling in incubation must report to the incubator pmc .


In [13]:
fpc1$id <- NULL
#fpc1$sid <- NULL
fpc1$index <- 0
for (i in 1:nrow(fpc1)) {
  fpc1[i, "index"] <- i-1
}


In [14]:
head(fpc1)
length(fpc1$sid)

sid,tid,word,pos,relation,word_source,CodeType,new_relations,new_word,new_pos,new_source,sid_text,index
0,0,ROOT,,NA,NA,1,NA,ROOT,,NA,ROOT each podling in incubation must report to the incubator pmc .,0
0,1,each,DT,det,podling,1,det,each,DT,podling,ROOT each podling in incubation must report to the incubator pmc .,1
0,2,podling,NN,nsubj,report,1,nsubj,podling,NN,report,ROOT each podling in incubation must report to the incubator pmc .,2
0,3,in,IN,case,incubation,1,case,in,IN,incubation,ROOT each podling in incubation must report to the incubator pmc .,3
0,4,incubation,NN,nmod,podling,1,nmod,incubation,NN,podling,ROOT each podling in incubation must report to the incubator pmc .,4
0,5,must,MD,aux,report,1,aux,must,MD,report,ROOT each podling in incubation must report to the incubator pmc .,5


[1] 21948

In [15]:
print(c(nrow(fpc1)-nrow(na.omit(fpc1)),max(fpc1$sid)+1))
assert_that( (nrow(fpc1) - nrow(na.omit(fpc1))) == max(fpc1$sid)+1)
if (FALSE | !preserve_old_analysis) {
    ### this gets rid of the ROOT elements, nothing more.  about 560: equal to the number of sentences
    fpc1 <- na.omit(fpc1)
}


[1] 886 886


[1] TRUE

In [16]:
head(fpc1)
length(unique(fpc1$index))

,sid,tid,word,pos,relation,word_source,CodeType,new_relations,new_word,new_pos,new_source,sid_text,index
2,0,1,each,DT,det,podling,1,det,each,DT,podling,ROOT each podling in incubation must report to the incubator pmc .,1
3,0,2,podling,NN,nsubj,report,1,nsubj,podling,NN,report,ROOT each podling in incubation must report to the incubator pmc .,2
4,0,3,in,IN,case,incubation,1,case,in,IN,incubation,ROOT each podling in incubation must report to the incubator pmc .,3
5,0,4,incubation,NN,nmod,podling,1,nmod,incubation,NN,podling,ROOT each podling in incubation must report to the incubator pmc .,4
6,0,5,must,MD,aux,report,1,aux,must,MD,report,ROOT each podling in incubation must report to the incubator pmc .,5
7,0,6,report,VB,root,ROOT,1,root,report,VB,ROOT,ROOT each podling in incubation must report to the incubator pmc .,6


[1] 21062

In [17]:
assert_that(length(fpc1$index) == length(unique(fpc1$index)))

# create lag variables
junktmp <- fpc1 
junktmp$sid <- NULL
junktmp$sid_text <- NULL
assert_that(length(junktmp$index) == length(unique(junktmp$index)))

lagjunk <- junktmp
lagjunk$tid <- junktmp$tid+1
lagjunk$index <- junktmp$index+1
colnames(lagjunk)[2:(ncol(lagjunk)-1)] <- paste0("before_", colnames(lagjunk)[2:(ncol(lagjunk)-1)])
if (FALSE | !preserve_old_analysis) {
    lagjunk <- na.omit(lagjunk)
}
assert_that(length(lagjunk$index) == length(unique(lagjunk$index)))
junk <- merge(junktmp, lagjunk, by=c("index", "tid"), all.x=T)
assert_that(length(junk$index) == length(unique(junk$index)))

[1] TRUE

[1] TRUE

[1] TRUE

[1] TRUE

In [18]:
head(junk)

index,tid,word,pos,relation,word_source,CodeType,new_relations,new_word,new_pos,new_source,before_word,before_pos,before_relation,before_word_source,before_CodeType,before_new_relations,before_new_word,before_new_pos,before_new_source
1,1,each,DT,det,podling,1,det,each,DT,podling,NA,NA,NA,NA,NA,NA,NA,NA,NA
2,2,podling,NN,nsubj,report,1,nsubj,podling,NN,report,each,DT,det,podling,1,det,each,DT,podling
3,3,in,IN,case,incubation,1,case,in,IN,incubation,podling,NN,nsubj,report,1,nsubj,podling,NN,report
4,4,incubation,NN,nmod,podling,1,nmod,incubation,NN,podling,in,IN,case,incubation,1,case,in,IN,incubation
5,5,must,MD,aux,report,1,aux,must,MD,report,incubation,NN,nmod,podling,1,nmod,incubation,NN,podling
6,6,report,VB,root,ROOT,1,root,report,VB,ROOT,must,MD,aux,report,1,aux,must,MD,report


In [19]:
head(fpc1)
nrow(fpc1)
length(unique(fpc1$index))

,sid,tid,word,pos,relation,word_source,CodeType,new_relations,new_word,new_pos,new_source,sid_text,index
2,0,1,each,DT,det,podling,1,det,each,DT,podling,ROOT each podling in incubation must report to the incubator pmc .,1
3,0,2,podling,NN,nsubj,report,1,nsubj,podling,NN,report,ROOT each podling in incubation must report to the incubator pmc .,2
4,0,3,in,IN,case,incubation,1,case,in,IN,incubation,ROOT each podling in incubation must report to the incubator pmc .,3
5,0,4,incubation,NN,nmod,podling,1,nmod,incubation,NN,podling,ROOT each podling in incubation must report to the incubator pmc .,4
6,0,5,must,MD,aux,report,1,aux,must,MD,report,ROOT each podling in incubation must report to the incubator pmc .,5
7,0,6,report,VB,root,ROOT,1,root,report,VB,ROOT,ROOT each podling in incubation must report to the incubator pmc .,6


[1] 21062

[1] 21062

In [20]:
# create lead variables
leadjunk <- junktmp
leadjunk$tid <- junktmp$tid-1
leadjunk$index <- junktmp$index-1
colnames(leadjunk)[2:(ncol(leadjunk)-1)] <- paste0("after_", colnames(leadjunk)[2:(ncol(leadjunk)-1)])

In [21]:
if (FALSE | !preserve_old_analysis) {
    leadjunk <- na.omit(leadjunk)
}
assert_that(length(leadjunk$index) == length(unique(leadjunk$index)))
assert_that(length(junk$index) == length(unique(junk$index)))


[1] TRUE

[1] TRUE

In [22]:
leadjunk

,tid,after_word,after_pos,after_relation,after_word_source,after_CodeType,after_new_relations,after_new_word,after_new_pos,after_new_source,index
2,0,each,DT,det,podling,1,det,each,DT,podling,0
3,1,podling,NN,nsubj,report,1,nsubj,podling,NN,report,1
4,2,in,IN,case,incubation,1,case,in,IN,incubation,2
5,3,incubation,NN,nmod,podling,1,nmod,incubation,NN,podling,3
6,4,must,MD,aux,report,1,aux,must,MD,report,4
7,5,report,VB,root,ROOT,1,root,report,VB,ROOT,5
8,6,to,IN,case,pmc,1,case,to,IN,pmc,6
9,7,the,DT,det,pmc,1,det,the,DT,pmc,7
10,8,incubator,NN,compound,pmc,1,compound,incubator,NN,pmc,8
11,9,pmc,NN,obl,report,1,obl,pmc,NN,report,9


In [23]:
junk <- merge(junk, leadjunk, by=c("index", "tid"), all.x=T)

head(junk)


index,tid,word,pos,relation,word_source,CodeType,new_relations,new_word,new_pos,...,before_new_source,after_word,after_pos,after_relation,after_word_source,after_CodeType,after_new_relations,after_new_word,after_new_pos,after_new_source
1,1,each,DT,det,podling,1,det,each,DT,...,NA,podling,NN,nsubj,report,1,nsubj,podling,NN,report
2,2,podling,NN,nsubj,report,1,nsubj,podling,NN,...,podling,in,IN,case,incubation,1,case,in,IN,incubation
3,3,in,IN,case,incubation,1,case,in,IN,...,report,incubation,NN,nmod,podling,1,nmod,incubation,NN,podling
4,4,incubation,NN,nmod,podling,1,nmod,incubation,NN,...,incubation,must,MD,aux,report,1,aux,must,MD,report
5,5,must,MD,aux,report,1,aux,must,MD,...,podling,report,VB,root,ROOT,1,root,report,VB,ROOT
6,6,report,VB,root,ROOT,1,root,report,VB,...,report,to,IN,case,pmc,1,case,to,IN,pmc


In [24]:
colnames(junk)

[1] "index"                "tid"                  "word"                
 [4] "pos"                  "relation"             "word_source"         
 [7] "CodeType"             "new_relations"        "new_word"            
[10] "new_pos"              "new_source"           "before_word"         
[13] "before_pos"           "before_relation"      "before_word_source"  
[16] "before_CodeType"      "before_new_relations" "before_new_word"     
[19] "before_new_pos"       "before_new_source"    "after_word"          
[22] "after_pos"            "after_relation"       "after_word_source"   
[25] "after_CodeType"       "after_new_relations"  "after_new_word"      
[28] "after_new_pos"        "after_new_source"

In [25]:
# create a series of dummy variable
junk <- junk[,c("CodeType", "tid", "new_relations", "new_word", "new_pos", "new_source", "before_new_relations", "before_new_word", "before_new_pos", "before_new_source", "after_new_relations", "after_new_word", "after_new_pos", "after_new_source")] #"sentiment", "before_sentiment", "after_sentiment")]
junk$new_word <- as.numeric(junk$new_word)
junk$new_source <- as.numeric(junk$new_source)
junk$before_new_word <- as.numeric(junk$before_new_word)
junk$before_new_source <- as.numeric(junk$before_new_source)
junk$after_new_word <- as.numeric(junk$after_new_word)
junk$after_new_source <- as.numeric(junk$after_new_source)

# Taking too long

In [26]:
junk <- dummy_cols(junk, select_columns = c("new_word", "before_new_word", "after_new_word", "new_relations", "new_pos", "before_new_relations", "before_new_pos", "after_new_relations", "after_new_pos", "before_new_source", "after_new_source"))

#colnames(junk)
# drop the original factor variable
junk <- junk[, -which(colnames(junk) %in% c("new_word", "new_source", "before_new_word", "tid","before_new_words", "before_new_source", "after_new_word", "after_new_source", "new_relations", "new_pos", "before_new_relations", "before_new_pos", "after_new_relations", "after_new_pos"))]
print(c(nrow(junk),nrow(fpc1)))



[1] 21062 21062


In [27]:
colnames(junk)

[1] "CodeType"                          "new_word_1"                       
    [3] "new_word_2"                        "new_word_3"                       
    [5] "new_word_4"                        "new_word_5"                       
    [7] "new_word_6"                        "new_word_7"                       
    [9] "new_word_9"                        "new_word_10"                      
   [11] "new_word_11"                       "new_word_12"                      
   [13] "new_word_13"                       "new_word_14"                      
   [15] "new_word_15"                       "new_word_16"                      
   [17] "new_word_17"                       "new_word_18"                      
   [19] "new_word_19"                       "new_word_20"                      
   [21] "new_word_21"                       "new_word_22"                      
   [23] "new_word_23"                       "new_word_24"                      
   [25] "new_word_25"                       "new_word_26"                      
   [27] "new_word_27"                       "new_word_28"                      
   [29] "new_word_29"                       "new_word_30"                      
   [31] "new_word_31"                       "new_word_32"                      
   [33] "new_word_33"                       "new_word_34"                      
   [35] "new_word_35"                       "new_word_36"                      
   [37] "new_word_37"                       "new_word_38"                      
   [39] "new_word_39"                       "new_word_40"                      
   [41] "new_word_41"                       "new_word_42"                      
   [43] "new_word_43"                       "new_word_44"                      
   [45] "new_word_45"                       "new_word_46"                      
   [47] "new_word_47"                       "new_word_48"                      
   [49] "new_word_49"                       "new_word_50"                      
   [51] "new_word_51"                       "new_word_52"                      
   [53] "new_word_53"                       "new_word_54"                      
   [55] "new_word_55"                       "new_word_56"                      
   [57] "new_word_57"                       "new_word_58"                      
   [59] "new_word_59"                       "new_word_60"                      
   [61] "new_word_61"                       "new_word_62"                      
   [63] "new_word_63"                       "new_word_64"                      
   [65] "new_word_65"                       "new_word_66"                      
   [67] "new_word_67"                       "new_word_68"                      
   [69] "new_word_69"                       "new_word_70"                      
   [71] "new_word_71"                       "new_word_72"                      
   [73] "new_word_73"                       "new_word_74"                      
   [75] "new_word_75"                       "new_word_76"                      
   [77] "new_word_77"                       "new_word_78"                      
   [79] "new_word_79"                       "new_word_80"                      
   [81] "new_word_81"                       "new_word_82"                      
   [83] "new_word_83"                       "new_word_84"                      
   [85] "new_word_85"                       "new_word_86"                      
   [87] "new_word_87"                       "new_word_88"                      
   [89] "new_word_89"                       "new_word_90"                      
   [91] "new_word_91"                       "new_word_92"                      
   [93] "new_word_93"                       "new_word_94"                      
   [95] "new_word_95"                       "new_word_96"                      
   [97] "new_word_97"                       "new_word_98"                      
   [99] "new_word_99"                       "new_word_100"                     
  [1

In [28]:
junk_test <- junk[-which(junk$CodeType != "1"),]
# junk_test$CodeType <- as.factor(junk_test$CodeType)

junk <- junk[-which(junk$CodeType == "1"),]
junk$CodeType <- as.factor(junk$CodeType)

df <- junk
df <- df[-which(df$CodeType == ""),]
if (preserve_old_analysis) {
    ### EXPERIMENT WITH NOT OMITTING NAS
    df <- na.omit(df)
}
df$CodeType <- as.numeric(df$CodeType)
df$CodeType <- df$CodeType -2

In [29]:
nrow(junk)
nrow(df)

[1] 14346

[1] 9500

In [30]:
sample <- fpc1[-which(fpc1$CodeType == "1"), c("new_word", "sid", "sid_text", "tid")]
# train <- junk[-which(junk$CodeType == "1"),]
sample$CodeType <- junk$CodeType
sample <- sample[-which(sample$CodeType == ""),]

if (preserve_old_analysis) {
    sample <- na.omit(sample)
}
assert_that(-1 %ni% sample$tid )
assert_that(all(dim(sample) == dim(na.omit(sample))))
assert_that(all(sample$CodeType == na.omit(sample)$CodeType))
if (preserve_old_analysis) {
    sample <- sample[rownames(df),]
}
sample$Code <- df$CodeType

[1] TRUE

[1] TRUE

[1] TRUE

In [31]:
### sanity check
assert_that(nrow(df) == nrow(sample))
if (preserve_old_analysis) {
    assert_that(
        ### for the other (pre experiment of removing NA's)
        all(df$CodeType == sample$Code) 
    )# confirm rows actually aligned right after all this
} else {
    assert_that(
        ### for one data format (post experiment of removing NA's)
        all(as.numeric(df$CodeType) == as.numeric(sample$CodeType)-2)
    ) # confirm rows actually aligned right after all this
}
all(as.numeric(df$CodeType) == as.numeric(sample$CodeType)-2)
all(df$CodeType == sample$Code)

[1] TRUE

[1] TRUE

[1] TRUE

[1] TRUE

In [32]:
#  prep for writing to file
sample[,1] <- as.character( sample[,1] )
sample$CodeType <- as.character( sample$CodeType )
sample$Code <- as.character( sample$Code )


In [33]:
head(sample)
nrow(sample)


,new_word,sid,sid_text,tid,CodeType,Code
7063,voting,324,ROOT the voting members of the board shall be comprised of residents and former residents of the city of camden .,2,Attribute,1
7064,members,324,ROOT the voting members of the board shall be comprised of residents and former residents of the city of camden .,3,Attribute,1
7065,of,324,ROOT the voting members of the board shall be comprised of residents and former residents of the city of camden .,4,Attribute,1
7066,the,324,ROOT the voting members of the board shall be comprised of residents and former residents of the city of camden .,5,Attribute,1
7067,board,324,ROOT the voting members of the board shall be comprised of residents and former residents of the city of camden .,6,Attribute,1
7068,shall,324,ROOT the voting members of the board shall be comprised of residents and former residents of the city of camden .,7,Deontic,3


[1] 9500

In [34]:
unique(sample$new_word)

[1] "voting"               "members"              "of"                  
   [4] "the"                  "board"                "shall"               
   [7] "be"                   "comprised"            "residents"           
  [10] "and"                  "former"               "city"                
  [13] "camden"               "nonvoting"            "employees"           
  [16] "officials"            "other"                "individuals"         
  [19] "who"                  "are"                  "to"                  
  [22] "serve"                "in"                   "an"                  
  [25] "advisory"             "capacity"             "consist"             
  [28] "five"                 "appointed"            "by"                  
  [31] "a"                    "majority"             "vote"                
  [34] "council"              "select"               "from"                
  [37] "list"                 "individual"           "candidates"          
  [40] "agreed"               "upon"                 "three"               
  [43] "member"               "two"                  "such"                
  [46] "designated"           "mayor"                "not"                 
  [49] "excluding"            "or"                   "chairperson"         
  [52] "terms"                "years"                "initial"             
  [55] "appointments"         "staggered"            "whereby"             
  [58] "half"                 "one-year"             "two-year"            
  [61] "advise"               "on"                   "following"           
  [64] "a."                   "building"             "strong"              
  [67] "sustainable"          "local"                "regional"            
  [70] "food"                 "systems"              "that"                
  [73] "ensure"               "communities"          "access"              
  [76] "affordable"           "nutritious"           "culturally"          
  [79] "appropriate"          "for"                  "all"                 
  [82] "at"                   "b."                   "development"         
  [85] "self-reliance"        "amongst"              "obtaining"           
  [88] "their"                "create"               "system"              
  [91] "growing"              "manufacturing"        "processing"          
  [94] "making"               "available"            "selling"             
  [97] "is"                   "locally"              "regionally"          
 [100] "based"                "grounded"             "principles"          
 [103] "good"                 "health"               "through"             
 [106] "periodically"         "meet"                 "report"              
 [109] "its"                  "activities"           "toward"              
 [112] "fulfillment"          "purpose"              "quorum"              
 [115] "membership"           "function"             "accordance"          
 [118] "with"                 "created"              "as"                  
 [121] "committee"            "foster"               "healthy"             
 [124] "supply"               "colorado"             "while"               
 [127] "enhancing"            "state"                "agricultural"        
 [130] "natural"              "resources"            "encouraging"         
 [133] "economic"             "growth"               "expanding"           
 [136] "viability"            "agriculture"          "improving"           
 [139] "our"                  "make"                 "recommendations"     
 [142] "general"              "assembly"             "regulatory"          
 [145] "agencies"             "policy"               "use"                 
 [148] "method"               "dialogue"             "consensus"           
 [151] "decision-making"      "arrive"               "consists"            
 [154] "thirteen"             "executive"            "director"            
 [157] "his"                  "her"                

In [35]:
indexes = c()
for (row in 1:nrow(sample)) {
    x <- gsub("[^[:alnum:]]", "", sample[row, "new_word"])
    if(x == ""){
        indexes <- c(indexes,row)
    }
}


In [40]:
sample[5147,]

,new_word,sid,sid_text,tid,CodeType,Code
15226,-,600,"ROOT such members shall represent the groups identified below : -LRB- 1 -RRB- business community - local business associations , restaurants , grocers associations , big box grocery stores , financial lenders/banks , greater louisville , inc. ; -LRB- 2 -RRB- community based organizations - farmers ' market managers , community gardens/gardeners , neighborhood associations , environmental groups , faith base community , concerned citizens and residents ; -LRB- 3 -RRB- rural and farm groups - regional farms/farmers , farm organizations ; -LRB- 4 -RRB- health and education - jefferson county public schools , universities , hospitals/healthcare agencies ; -LRB- 5 -RRB- neighborhoods experiencing the `` food desert '' phernomenon , including 40203 , 40210 , 40211 , and 40212 zip codes",15,Condition,2


In [36]:
indexes

[1] 2058 2066 4942 5147

In [41]:
df <- df[-indexes,]
sample <- sample[-indexes,]

print("final sizes")
print(c("sample:", dim(sample)))
print(c("df:", dim(df)))

write.csv(sample, "/Users/abhijit/IGRI/data//step1_data_sample.csv", row.names=FALSE)
write.csv(df, "/Users/abhijit/IGRI/data//step1_data_df.csv", row.names=FALSE)



[1] "final sizes"
[1] "sample:" "9496"    "6"      
[1] "df:"   "9496"  "10979"


In [42]:
#generate test features
df_test <- junk_test
df_test$CodeType <- 1
sample_test <- fpc1[which(fpc1$CodeType == "1"), c("new_word", "sid", "sid_text", "tid")]
assert_that(-1 %ni% sample_test$tid )
assert_that(all(dim(sample_test) == dim(na.omit(sample_test))))


indexes = c()
for (row in 1:nrow(sample_test)) {
    x <- gsub("[^[:alnum:]]", "", sample_test[row, "new_word"])
    if(x == ""){
        indexes <- c(indexes,row)
    }
}
df_test <- df_test[-indexes,]
sample_test <- sample_test[-indexes,]


sample_test$CodeType <- ""
sample_test$Code <- ""

print("final sizes")
print(c("sample:", dim(sample_test)))
print(c("df:", dim(df_test)))

write.csv(sample_test, "/Users/abhijit/IGRI/data//asf_data_sample.csv", row.names=FALSE)
write.csv(df_test, "/Users/abhijit/IGRI/data//asf_data_df.csv", row.names=FALSE)

[1] TRUE

[1] TRUE

[1] "final sizes"
[1] "sample:" "6010"    "6"      
[1] "df:"   "6010"  "10979"
